# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

*toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
import numpy as np
from pymystem3 import Mystem
import nltk
from nltk.corpus import stopwords as nltk_stopwords
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, roc_auc_score, roc_curve
from sklearn.utils import shuffle
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('/datasets/toxic_comments.csv')
df.head(10)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


Работать будем с английскими комментариями, видим, что есть разница в регистрах, учтем

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Пропусков нет - супер! Типы данных адекватные

In [4]:
df['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

видим дисбаланс классов, это надо будет учесть при обучении модели

In [5]:
m = Mystem()

def lemmatize_text(text):
    text = text.lower()
    lemm_text = "".join(m.lemmatize(text))
    clear = re.sub(r'[^a-zA-Z]', ' ', lemm_text) 
    return " ".join(clear.split())

In [6]:
%%time

df['lemm'] = df['text'].apply(lemmatize_text)

CPU times: user 40.1 s, sys: 7.74 s, total: 47.8 s
Wall time: 1min 41s


In [7]:
df.isna().sum()

text     0
toxic    0
lemm     0
dtype: int64

просто на всякий случай проверил пропуски после лемматизации и добавления новой колонки

In [8]:
df = df.drop(['text'], axis=1)

эта колонка нам больше не нужна, дальше будем работать с леммами

In [9]:
df2 = df.copy()

перед тем, как проводить преобразования создаю копию, чтобы можно было в случае чего быстро вернуться к исходной таблице

In [10]:
df2.shape

(159571, 2)

In [11]:
df.head()

,toxic,lemm
0,0,explanation why the edits made under my userna...
1,0,d aww he matches this background colour i m se...
2,0,hey man i m really not trying to edit war it s...
3,0,more i can t make any real suggestions on impr...
4,0,you sir are my hero any chance you remember wh...


In [12]:
features_train, features_test, target_train, target_test = train_test_split(df['lemm'], 
                                                                              df['toxic'], 
                                                                              test_size=0.2, 
                                                                              random_state=12345)


print(features_train.shape)
print(features_test.shape)
print(target_train.shape)
print(target_test.shape)

(127656,)
(31915,)
(127656,)
(31915,)


### Создаем TF-IDF

In [13]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [14]:
stopwords = set(nltk_stopwords.words('english'))

tf_idf = TfidfVectorizer(stop_words=stopwords)

In [15]:
features_train = tf_idf.fit_transform(features_train.values.astype('U'))
features_test = tf_idf.transform(features_test.values.astype('U'))

я не стал создавать валидационную выборку, потому что как такового подбора параметров скорее всего делать не придется

## Обучение

### LogisticRegression

In [16]:
weight = df['toxic'].value_counts()[0] / df['toxic'].value_counts()[1]

weight

8.834884437596301

посчитал соотношение классов, чтобы можно было сбалансировать через словарик

In [18]:
%%time

dict_class = {0:1, 1:weight}

log_model2 = LogisticRegression(class_weight = dict_class)
log_model2.fit(features_train, target_train)

log2_predict = log_model2.predict(features_test)
log2_f1 = f1_score(target_test, log2_predict)

print('F1 log_model2:', log2_f1)

F1 log_model2: 0.7575465196416264
CPU times: user 8.13 s, sys: 7.23 s, total: 15.4 s
Wall time: 15.4 s


### CatBoostClassifier

In [20]:
%%time

cat_model2 = CatBoostClassifier(verbose=False, iterations=200, class_weights=[1, 8])
cat_model2.fit(features_train, target_train)
target_predict = cat_model2.predict(features_test)
cv_cat2 = cross_val_score(cat_model2,
                         features_train,
                         target_train, 
                         cv=3,
                         scoring='f1').mean()

test2_f1 = f1_score(target_test, target_predict)
print('F1 на cv', cv_cat2)
print('F1 на тесте', test2_f1)

F1 на cv 0.7548799786670456
F1 на тесте 0.7489325362937659
CPU times: user 42min 20s, sys: 51.6 s, total: 43min 11s
Wall time: 43min 17s


## Выводы

In [21]:
final = pd.DataFrame({'model' : ['CatBoost', 'LogisticRegression'], 
                      'F1' : [test2_f1, log2_f1]})

final

,model,F1
0,CatBoost,0.748933
1,LogisticRegression,0.757547


1. Данные подготовлены
2. Выбраны модели для обучения и методы балансировки классов
3. После TF-IDF создалось большое количество фич, что потребовало большого количества времени для обучения модели катбуст. Логистическая регрессия в свою очередь обучилась достаточно быстро, при этом показала более качественный результат.

В данном проекте себя лучше показала логистическая регрессия, но справедливости ради стоит уточнить, что я не проводил каких-либо подборов параметров для катбуста, поэтому я уверен, что покрутив параметры через гридсерч мы могли бы получить более качественную метрику f1.